In [1]:
import numpy
import copy

In [4]:
class FD:
    nodes = [] # All the possible combination of the attributes
    graph = {} # graph representation of FDs
    sdf = {} # standard FDs
    attr = None    # list of attributes in relation
    N = 0          # Total number of attributes
    CK = []        # candidate keys
    def __init__(self, N):
        self.N = N
        # create alphabetical attribute names
        first_char = ord('A')
        self.attr =[chr(i) for i in xrange(first_char, first_char+N)]
        # create all subsets of attributes
        self.nodes = self.genNodes(copy.copy(self.attr))
        self.nodes.remove('')
        print self.nodes
        # initialize the set of FD
        self.init()
        print self.graph
        # find full closure of FD
        self.fullClosure()
        print self.graph
        # find minimum cover of FD
        self.minimal()
        # find candidate keys
        self.candidateKeys()
        print self.CK
    
    # add random FD in the beginning
    def init(self):
        # adding trivial dependencies
        for n in self.nodes:
            self.graph[n] = []
        for n in self.nodes:
            subsets = self.genNodes(list(n))
            subsets.remove('')
            for s in subsets:
                self.graph[n].append(s)
        # TODO add random FD
        #AB->D, BD->C, CD->A, and AD->B
        self.graph['AB'].append('D')
        self.graph['BD'].append('C')
        self.graph['CD'].append('A')
        self.graph['AD'].append('B')
        
        
    # generate the all the possible left side 2^n attribute
    def genNodes(self, attr_left):
        if len(attr_left)==0:
            return ['']
        current = attr_left.pop()
        new_res = self.genNodes(attr_left)
        with_attr = []
        for x in new_res:
            addition = ''.join(sorted(list(set(list(x+current)))))
            with_attr.append(addition)
        return new_res+with_attr
    
    # dfs for finding all transitive FD
    def dfs(self, node):
        Open = [node]
        Closed = []
        while Open != []:
            current = Open.pop()
            for u in self.graph[current]:
                if u==current:
                    continue
                if not(u in Open) and not(u in Closed):
                    Open.append(u)
            Closed.append(current)
        return Closed
    # use transitivity axiom
    def transit(self):
        for n in self.nodes:
            reach = self.dfs(n)
            for r in reach:
                if not(r in self.graph[n]):
                    self.graph[n].append(r)
    # use augmentation axiom
    def augment(self):
        for n in self.nodes:
            total = []
            for a in self.graph[n]:
                total += list(a)
            total = list(set(total))
            res = self.genNodes(total)
            res.remove('')
            self.graph[n] = res
    # finds the set of full closures for the input FD                       
    def fullClosure(self):
        print 'full closure'
        self.augment()
        self.transit()
        self.augment()
    # finding all candidate keys
    def candidateKeys(self):
        print "candidate keys"
        self.CK = []
        for node in self.graph:
            if ''.join(sorted(self.attr)) in self.graph[node]:
                self.CK.append(node)
        super_keys = []
        for ck1 in self.CK:
            for ck2 in self.CK:
                if ck1 != ck2 and set(ck1).issubset(set(ck2)):
                    super_keys.append(ck2)
        for sk in set(super_keys):
            self.CK.remove(sk)
        return self.CK
                
    # delete redendant FD's
    def delRedundant(self):
        redundants = []
        for node in self.sdf:
            subsets = self.genNodes(list(node))
            subsets.remove('')
            subsets.remove(node)
            for s in subsets:
                if s in self.sdf:
                    if set(self.sdf[s]) == set(self.sdf[node]):
                        redundants.append(node)
                        break
        for r in redundants:
            self.sdf.pop(r, None)
            
    # minimize left sides of standard FDs
    def minimizeLeft(self):
        redundants = []
        for node in self.sdf:
            for a in self.sdf[node]:
                b = ''.join(sorted(node+a))
                if b in self.sdf:
                    redundants.append(b)
        for r in redundants:
            self.sdf.pop(r, None)    
        self.standardize(self.sdf, full=True)
    # create the set of standardize FDs
    def standardize(self, inputFDDict, full=False):
        for node in inputFDDict:
            stan_list = []
            for e in inputFDDict[node]:
                non_standard = set(list(e))
                standard = non_standard.difference(set(node))
                standard = sorted(list(standard))
                if not(full):
                    standard = ''.join(standard)
                if not(standard==''):
                    if full:
                        stan_list += standard
                    else:
                        stan_list.append(standard)
            stan_list = list(set(stan_list))
            if stan_list != []:
                self.sdf[node] = stan_list
    # find the minimal set of FD
    def minimal(self):
        print "minimal cover"
        self.standardize(self.graph)
        print self.sdf
        self.minimizeLeft()
        print self.sdf
        self.delRedundant()
        print self.sdf
                

In [5]:
FD(4)

['A', 'B', 'AB', 'C', 'AC', 'BC', 'ABC', 'D', 'AD', 'BD', 'ABD', 'CD', 'ACD', 'BCD', 'ABCD']
{'A': ['A'], 'BD': ['B', 'D', 'BD', 'C'], 'C': ['C'], 'B': ['B'], 'ABD': ['A', 'B', 'AB', 'D', 'AD', 'BD', 'ABD'], 'D': ['D'], 'ABCD': ['A', 'B', 'AB', 'C', 'AC', 'BC', 'ABC', 'D', 'AD', 'BD', 'ABD', 'CD', 'ACD', 'BCD', 'ABCD'], 'BC': ['B', 'C', 'BC'], 'BCD': ['B', 'C', 'BC', 'D', 'BD', 'CD', 'BCD'], 'ACD': ['A', 'C', 'AC', 'D', 'AD', 'CD', 'ACD'], 'AC': ['A', 'C', 'AC'], 'ABC': ['A', 'B', 'AB', 'C', 'AC', 'BC', 'ABC'], 'CD': ['C', 'D', 'CD', 'A'], 'AB': ['A', 'B', 'AB', 'D'], 'AD': ['A', 'D', 'AD', 'B']}
full closure
{'A': ['A'], 'BD': ['A', 'C', 'AC', 'B', 'AB', 'BC', 'ABC', 'D', 'AD', 'CD', 'ACD', 'BD', 'ABD', 'BCD', 'ABCD'], 'C': ['C'], 'B': ['B'], 'ABD': ['A', 'C', 'AC', 'B', 'AB', 'BC', 'ABC', 'D', 'AD', 'CD', 'ACD', 'BD', 'ABD', 'BCD', 'ABCD'], 'D': ['D'], 'ABCD': ['A', 'C', 'AC', 'B', 'AB', 'BC', 'ABC', 'D', 'AD', 'CD', 'ACD', 'BD', 'ABD', 'BCD', 'ABCD'], 'BC': ['C', 'B', 'BC'], 'BCD': 

<__main__.FD instance at 0x051D97D8>

In [57]:
x = [1, 2, 3]
x == []

False